In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data = pd.read_csv('prelim.csv', low_memory=False)

In [ ]:
len(data)

116426

In [ ]:
data.head()

,Unnamed: 0,SUBJECT_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,DIAGNOSIS,GENDER,DOB,DOD,DOD_HOSP,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,LOS,MORTALITY
0,0,22,2196-04-09 12:26:00,2196-04-10 15:54:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,BENZODIAZEPINE OVERDOSE,F,2131-05-07,NaN,NaN,MICU,MICU,52,52,2196-04-09 12:27:00,2196-04-10 15:54:00,1.1438,0
1,1,23,2153-09-03 07:15:00,2153-09-08 19:10:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,MARRIED,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,M,2082-07-17,NaN,NaN,CSRU,CSRU,14,14,2153-09-03 09:38:55,2153-09-04 15:59:11,1.2641,0
2,2,23,2153-09-03 07:15:00,2153-09-08 19:10:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,MARRIED,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,M,2082-07-17,NaN,NaN,SICU,SICU,57,57,2157-10-21 11:40:38,2157-10-22 16:08:48,1.1862,0
3,3,23,2157-10-18 19:34:00,2157-10-25 14:00:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,BRAIN MASS,M,2082-07-17,NaN,NaN,CSRU,CSRU,14,14,2153-09-03 09:38:55,2153-09-04 15:59:11,1.2641,0
4,4,23,2157-10-18 19:34:00,2157-10-25 14:00:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,BRAIN MASS,M,2082-07-17,NaN,NaN,SICU,SICU,57,57,2157-10-21 11:40:38,2157-10-22 16:08:48,1.1862,0


`SUBJECT_ID`: an integer number identifying a particular patient. This can be thought of as a substitute for a unique medical record number. In the flat file data posted on PhysioNet, the number representing the Subject ID is left padded with zeros to five digits and preceded by the letter s. In the relational database, the Subject ID has no preceding
letter or leading zeros.

`ADMITTIME`: time stamp admitted into the hospital

`DISCHTIME`: time stamp discharged out of the hospital

`DEATHTIME`: time stamp of death

 `ADMISSION_TYPE`: admission type

`ADMISSION_LOCATION`: admission room location

 `DISCHARGE_LOCATION`: discharge location

 `INSURANCE`: insurance type

 `LANGUAGE` : used language

`RELIGION`: religion

`MARITAL_STATUS`: marital status

 `ETHNICITY`: ethinicity

  `DIAGNOSIS'`: diagonosis

  `MORTALITY`:  binary, has value 1 if deathtime is not NaN

`GENDER` : gender

`DOB` : time stamp, date of birth

`DOD` : time stamp, date of death (maybe unknown to hospital)

`DOD_HOSP`: time stamp, date of death in the hospital

 `DBSOURCE'`: data base source, can ignore

 
 `FIRST_CAREUNIT`: first care unit type stayed

`LAST_CAREUNIT`: last care unit type stayed

 `FIRST_WARDID`: first wardid had, number

  `LAST_WARDID`: last wardid had, number

   `INTIME'`: time stamp into the ICU

   `OUTTIME`: time stamp out of the ICU

  `LOS`: length stayed in the ICU, in day

In [ ]:
quasi_lists = ['ADMISSION_TYPE', 'ADMISSION_LOCATION', 'DISCHARGE_LOCATION',
       'INSURANCE', 'LANGUAGE', 'RELIGION', 'MARITAL_STATUS', 'ETHNICITY',
       'DIAGNOSIS', 'GENDER', 'FIRST_CAREUNIT',
       'LAST_CAREUNIT', 'FIRST_WARDID', 'LAST_WARDID',
       'LOS']

In [ ]:
data[quasi_lists].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116426 entries, 0 to 116425
Data columns (total 15 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   ADMISSION_TYPE      116426 non-null  object 
 1   ADMISSION_LOCATION  116426 non-null  object 
 2   DISCHARGE_LOCATION  116426 non-null  object 
 3   INSURANCE           116426 non-null  object 
 4   LANGUAGE            76639 non-null   object 
 5   RELIGION            115901 non-null  object 
 6   MARITAL_STATUS      105497 non-null  object 
 7   ETHNICITY           116426 non-null  object 
 8   DIAGNOSIS           116371 non-null  object 
 9   GENDER              116426 non-null  object 
 10  FIRST_CAREUNIT      116426 non-null  object 
 11  LAST_CAREUNIT       116426 non-null  object 
 12  FIRST_WARDID        116426 non-null  int64  
 13  LAST_WARDID         116426 non-null  int64  
 14  LOS                 116414 non-null  float64
dtypes: float64(1), int64(2), object(12

In [ ]:
data_fillna = data.fillna(-999)

To identify which columns we need to remove, we plan to use the groupby function in Python. However, groupby function would ignore NaN values when using variables as groups. To ensure we get correct result, we first decide to replace NaN values as -999, which is a numerical value that has not been seen in our dataframe.

# K-Anonmity

In [ ]:
# supression
def supression(data, quasi_set, k):
    len_remove = 0
    lst_remove = []
    grouped = data.groupby(by=quasi_set)
    for name, group in grouped:
        if group.shape[0] < k:
            len_remove += group.shape[0]
            lst_remove.extend(list(group.index))
    return len_remove, lst_remove

In [ ]:
len_remove, lst_remove = supression(data_fillna, quasi_lists, 3)
len_remain = data_fillna.shape[0] - len_remove
print(str(len_remain) + " records of the original dataset remain if we use suppression to make it 3-anonymous")
data_drop_3 = data_fillna.drop(lst_remove, axis=0)

3265 records of the original dataset remain if we use suppression to make it 3-anonymous


In [ ]:
data_drop_3.head()

,Unnamed: 0,SUBJECT_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,DIAGNOSIS,GENDER,DOB,DOD,DOD_HOSP,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,LOS,MORTALITY
257,257,109,2140-01-19 13:25:00,2140-01-21 13:25:00,-999,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Medicaid,ENGL,NOT SPECIFIED,SINGLE,BLACK/AFRICAN AMERICAN,HYPERTENSION,F,2117-08-07,2142-08-30,2142-08-30,MICU,MICU,12,12,2137-11-04 19:37:58,2137-11-05 17:04:39,0.8935,1
258,258,109,2140-01-19 13:25:00,2140-01-21 13:25:00,-999,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Medicaid,ENGL,NOT SPECIFIED,SINGLE,BLACK/AFRICAN AMERICAN,HYPERTENSION,F,2117-08-07,2142-08-30,2142-08-30,MICU,MICU,15,15,2137-11-09 13:31:16,2137-11-12 17:56:27,3.1842,1
259,259,109,2140-01-19 13:25:00,2140-01-21 13:25:00,-999,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Medicaid,ENGL,NOT SPECIFIED,SINGLE,BLACK/AFRICAN AMERICAN,HYPERTENSION,F,2117-08-07,2142-08-30,2142-08-30,MICU,MICU,12,12,2137-11-14 20:47:58,2137-11-20 22:38:41,6.0769,1
260,260,109,2140-01-19 13:25:00,2140-01-21 13:25:00,-999,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Medicaid,ENGL,NOT SPECIFIED,SINGLE,BLACK/AFRICAN AMERICAN,HYPERTENSION,F,2117-08-07,2142-08-30,2142-08-30,MICU,MICU,15,15,2138-04-16 17:45:30,2138-04-19 18:00:22,3.0103,1
261,261,109,2140-01-19 13:25:00,2140-01-21 13:25:00,-999,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Medicaid,ENGL,NOT SPECIFIED,SINGLE,BLACK/AFRICAN AMERICAN,HYPERTENSION,F,2117-08-07,2142-08-30,2142-08-30,SICU,SICU,57,57,2140-01-19 13:25:51,2140-01-21 13:20:11,1.9961,1


In [ ]:
len_remove, lst_remove = supression(data_fillna, quasi_lists, 5)
len_remain = data_fillna.shape[0] - len_remove
print(str(len_remain) + " records of the original dataset remain if we use suppression to make it 5-anonymous")
data_drop_5 = data_fillna.drop(lst_remove, axis=0)

1850 records of the original dataset remain if we use suppression to make it 5-anonymous


In [ ]:
data_drop_5.head()

,Unnamed: 0,SUBJECT_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,DIAGNOSIS,GENDER,DOB,DOD,DOD_HOSP,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,LOS,MORTALITY
257,257,109,2140-01-19 13:25:00,2140-01-21 13:25:00,-999,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Medicaid,ENGL,NOT SPECIFIED,SINGLE,BLACK/AFRICAN AMERICAN,HYPERTENSION,F,2117-08-07,2142-08-30,2142-08-30,MICU,MICU,12,12,2137-11-04 19:37:58,2137-11-05 17:04:39,0.8935,1
258,258,109,2140-01-19 13:25:00,2140-01-21 13:25:00,-999,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Medicaid,ENGL,NOT SPECIFIED,SINGLE,BLACK/AFRICAN AMERICAN,HYPERTENSION,F,2117-08-07,2142-08-30,2142-08-30,MICU,MICU,15,15,2137-11-09 13:31:16,2137-11-12 17:56:27,3.1842,1
259,259,109,2140-01-19 13:25:00,2140-01-21 13:25:00,-999,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Medicaid,ENGL,NOT SPECIFIED,SINGLE,BLACK/AFRICAN AMERICAN,HYPERTENSION,F,2117-08-07,2142-08-30,2142-08-30,MICU,MICU,12,12,2137-11-14 20:47:58,2137-11-20 22:38:41,6.0769,1
260,260,109,2140-01-19 13:25:00,2140-01-21 13:25:00,-999,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Medicaid,ENGL,NOT SPECIFIED,SINGLE,BLACK/AFRICAN AMERICAN,HYPERTENSION,F,2117-08-07,2142-08-30,2142-08-30,MICU,MICU,15,15,2138-04-16 17:45:30,2138-04-19 18:00:22,3.0103,1
261,261,109,2140-01-19 13:25:00,2140-01-21 13:25:00,-999,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Medicaid,ENGL,NOT SPECIFIED,SINGLE,BLACK/AFRICAN AMERICAN,HYPERTENSION,F,2117-08-07,2142-08-30,2142-08-30,SICU,SICU,57,57,2140-01-19 13:25:51,2140-01-21 13:20:11,1.9961,1


# Synthetic

In [ ]:
# adding synthetic records
def synthetic_records(data, quasi_set, k):
    len_add = 0
    grouped = data.groupby(by=quasi_set)
    for name, group in grouped:
        if group.shape[0] < k:
            len_add +=  (k - group.shape[0])
    return len_add

In [ ]:
synthetic_records(data_fillna, quasi_lists, 3)

219719

This shows we have to add 219899 records to the original MIMIC dataset to make it 3-anonymous.

In [ ]:
synthetic_records(data_fillna, quasi_lists, 5)

442414

This shows we have to add 442714 records to the original MIMIC dataset to make it 5-anonymous.



# Generalization or Blurring

In [ ]:
quasi_df = data_fillna[quasi_lists]
quasi_df.head()

,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,DIAGNOSIS,GENDER,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,LOS
0,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,-999,UNOBTAINABLE,MARRIED,WHITE,BENZODIAZEPINE OVERDOSE,F,MICU,MICU,52,52,1.1438
1,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,-999,CATHOLIC,MARRIED,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,M,CSRU,CSRU,14,14,1.2641
2,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,-999,CATHOLIC,MARRIED,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,M,SICU,SICU,57,57,1.1862
3,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,BRAIN MASS,M,CSRU,CSRU,14,14,1.2641
4,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,BRAIN MASS,M,SICU,SICU,57,57,1.1862


In [ ]:
quasi_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116426 entries, 0 to 116425
Data columns (total 15 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   ADMISSION_TYPE      116426 non-null  object 
 1   ADMISSION_LOCATION  116426 non-null  object 
 2   DISCHARGE_LOCATION  116426 non-null  object 
 3   INSURANCE           116426 non-null  object 
 4   LANGUAGE            116426 non-null  object 
 5   RELIGION            116426 non-null  object 
 6   MARITAL_STATUS      116426 non-null  object 
 7   ETHNICITY           116426 non-null  object 
 8   DIAGNOSIS           116426 non-null  object 
 9   GENDER              116426 non-null  object 
 10  FIRST_CAREUNIT      116426 non-null  object 
 11  LAST_CAREUNIT       116426 non-null  object 
 12  FIRST_WARDID        116426 non-null  int64  
 13  LAST_WARDID         116426 non-null  int64  
 14  LOS                 116426 non-null  float64
dtypes: float64(1), int64(2), object(12

In [ ]:
quasi_df.columns

Index(['ADMISSION_TYPE', 'ADMISSION_LOCATION', 'DISCHARGE_LOCATION',
       'INSURANCE', 'LANGUAGE', 'RELIGION', 'MARITAL_STATUS', 'ETHNICITY',
       'DIAGNOSIS', 'GENDER', 'FIRST_CAREUNIT', 'LAST_CAREUNIT',
       'FIRST_WARDID', 'LAST_WARDID', 'LOS'],
      dtype='object')

In [ ]:
df = data_fillna.copy()

In [ ]:
categorical = ['ADMISSION_TYPE', 'ADMISSION_LOCATION', 'DISCHARGE_LOCATION',
       'INSURANCE', 'LANGUAGE', 'RELIGION', 'MARITAL_STATUS', 'ETHNICITY',
       'DIAGNOSIS', 'GENDER', 'FIRST_CAREUNIT',
       'LAST_CAREUNIT', 'FIRST_WARDID', 'LAST_WARDID']

numerical = ['LOS']

In [ ]:
for name in categorical:
    df[name] = df[name].astype('category')

Next we take a look at spans (max-min for numerical columns, number of different values for categorical columns) of all columns for a partition of a dataframe.

In [ ]:
## spans (max-min for numerical columns, number of different values for categorical columns) 
## of all columns for a partition of a dataframe.
def get_spans(df, partition, scale=None):
    spans = {}
    for column in df.columns:
        if column in categorical:
            span = len(df[column][partition].unique())
        elif column in numerical:
            span = df[column][partition].max()-df[column][partition].min()
        else:
            continue
        if scale is not None:
            span = span/scale[column]
        spans[column] = span
    return spans

In [ ]:
full_spans = get_spans(df, df.index)
print(full_spans)

{'ADMISSION_TYPE': 4, 'ADMISSION_LOCATION': 9, 'DISCHARGE_LOCATION': 17, 'INSURANCE': 5, 'LANGUAGE': 76, 'RELIGION': 21, 'MARITAL_STATUS': 8, 'ETHNICITY': 41, 'DIAGNOSIS': 15669, 'GENDER': 2, 'FIRST_CAREUNIT': 6, 'LAST_CAREUNIT': 6, 'FIRST_WARDID': 16, 'LAST_WARDID': 17, 'LOS': 1172.0725}


Next, we take a look at all categorical variables, and check to see whether we should generalize or delete them, or leave it there.

In [ ]:
for i in categorical:
    counts = df[i].value_counts()
    filtered = counts[counts <= 5]
    print(counts)
    print("number of values with <5 counts is %i"%len(filtered))
    print("=================================")

EMERGENCY    94128
ELECTIVE     11905
NEWBORN       8116
URGENT        2277
Name: ADMISSION_TYPE, dtype: int64
number of values with <5 counts is 0
EMERGENCY ROOM ADMIT         56752
CLINIC REFERRAL/PREMATURE    23466
PHYS REFERRAL/NORMAL DELI    21322
TRANSFER FROM HOSP/EXTRAM    13637
TRANSFER FROM SKILLED NUR      671
TRANSFER FROM OTHER HEALT      246
** INFO NOT AVAILABLE **       212
HMO REFERRAL/SICK              105
TRSF WITHIN THIS FACILITY       15
Name: ADMISSION_LOCATION, dtype: int64
number of values with <5 counts is 0
HOME                         33848
HOME HEALTH CARE             29185
SNF                          16591
REHAB/DISTINCT PART HOSP     13248
DEAD/EXPIRED                  9348
LONG TERM CARE HOSPITAL       6728
SHORT TERM HOSPITAL           1926
LEFT AGAINST MEDICAL ADVI     1923
DISC-TRAN CANCER/CHLDRN H     1395
HOSPICE-HOME                   726
DISCH-TRAN TO PSYCH HOSP       719
HOSPICE-MEDICAL FACILITY       253
HOME WITH HOME IV PROVIDR      191
ICF   

In [ ]:
categorical_blurring = ['DIAGNOSIS','LANGUAGE']

In [ ]:
# blurring (column suppression, removing whole columns that don't fit for k-anonymity) 
def blurring(df, col):
    result_df = df.drop(columns=[col])
    return result_df

In [ ]:
# generalization (changing column values to be more general such as using an age range instead of a specific age)
def generalize(df, col, partition = 4):
    result_df = df.copy()

    ## if col is numerical, generalize them into sub-bins by quantiles
    if col in numerical: 
        span = max(result_df[col]) - 0
        interval = 1/partition
        partition_lst = [i*interval*span for i in range(partition)]
        label_lst = ['q'+str(i) for i in range(partition)]
        pd_cut_full_lst, label_full_lst = [-1000], ["q1-"]
        pd_cut_full_lst += partition_lst
        label_full_lst += label_lst
        pd_cut_full_lst.append(max(result_df[col])+1)
        result_df[str(col+"_percentile")] = pd.cut(result_df[col], pd_cut_full_lst, labels = label_full_lst).astype(str)
    
    ## last step: remove original column
    result_df = result_df.drop(columns=[col])
    return result_df

In [ ]:
def naive_search(col_lst, df):
    remove_dict = {}
    total_completion_rate_dict = {}
    for i in col_lst:
        quasi_lists_copy = quasi_lists.copy()
        if i in numerical:
            temp_df = generalize(df, i)
            # new quasi_lists after generalize
            quasi_lists_copy.remove(i)
            quasi_lists_copy.append(str(i+"_percentile"))
            len_remove, lst_remove = supression(temp_df, quasi_lists_copy, 5)
            print("if generalize on "+ i + " we need to remove "+ str(len_remove) + " records.")
        elif i in categorical_blurring:
            temp_df = blurring(df, i)
            # new quasi_lists after blurring
            quasi_lists_copy.remove(i)
            len_remove, lst_remove = supression(temp_df, quasi_lists_copy, 5)
            print("if blurring on "+ i + " we need to remove "+ str(len_remove) + " records.")
        data_drop= temp_df.drop(lst_remove, axis=0)
        remove_dict[i] = len_remove
        print("===========================================")
    return temp_df

In [ ]:
search_lst = categorical_blurring + numerical
search_lst

['DIAGNOSIS', 'LANGUAGE', 'LOS']

In [ ]:
naive_search(search_lst, df)

if blurring on DIAGNOSIS we need to remove 106281 records.
if blurring on LANGUAGE we need to remove 114541 records.
if generalize on LOS we need to remove 99014 records.


,Unnamed: 0,SUBJECT_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,DIAGNOSIS,GENDER,DOB,DOD,DOD_HOSP,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,MORTALITY,LOS_percentile
0,0,22,2196-04-09 12:26:00,2196-04-10 15:54:00,-999,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,-999,UNOBTAINABLE,MARRIED,WHITE,BENZODIAZEPINE OVERDOSE,F,2131-05-07,-999,-999,MICU,MICU,52,52,2196-04-09 12:27:00,2196-04-10 15:54:00,0,q0
1,1,23,2153-09-03 07:15:00,2153-09-08 19:10:00,-999,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,-999,CATHOLIC,MARRIED,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,M,2082-07-17,-999,-999,CSRU,CSRU,14,14,2153-09-03 09:38:55,2153-09-04 15:59:11,0,q0
2,2,23,2153-09-03 07:15:00,2153-09-08 19:10:00,-999,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,-999,CATHOLIC,MARRIED,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,M,2082-07-17,-999,-999,SICU,SICU,57,57,2157-10-21 11:40:38,2157-10-22 16:08:48,0,q0
3,3,23,2157-10-18 19:34:00,2157-10-25 14:00:00,-999,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,BRAIN MASS,M,2082-07-17,-999,-999,CSRU,CSRU,14,14,2153-09-03 09:38:55,2153-09-04 15:59:11,0,q0
4,4,23,2157-10-18 19:34:00,2157-10-25 14:00:00,-999,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,BRAIN MASS,M,2082-07-17,-999,-999,SICU,SICU,57,57,2157-10-21 11:40:38,2157-10-22 16:08:48,0,q0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116421,116421,98794,2127-11-07 11:00:00,2127-11-11 14:30:00,-999,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,AORTIC STENOSIS\AORTIC VALVE REPLACEMENT /SDA,M,2049-07-29,-999,-999,CSRU,CSRU,15,15,2127-11-07 10:02:54,2127-11-09 16:58:46,0,q0
116422,116422,98797,2132-12-24 20:06:00,2132-12-25 12:00:00,2132-12-25 12:00:00,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,Medicare,ENGL,CATHOLIC,WIDOWED,WHITE,ALTERED MENTAL STATUS,M,2044-12-27,2132-12-25,2132-12-25,MICU,MICU,50,50,2132-12-24 20:08:16,2132-12-26 01:51:14,1,q0
116423,116423,98800,2131-03-30 21:13:00,2131-04-02 15:02:00,-999,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Private,ENGL,NOT SPECIFIED,SINGLE,WHITE,TRAUMA,F,2111-11-05,-999,-999,TSICU,TSICU,14,14,2131-03-30 21:14:14,2131-03-31 18:18:14,0,q0
116424,116424,98802,2151-03-05 20:00:00,2151-03-06 09:10:00,2151-03-06 09:10:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,DEAD/EXPIRED,Medicare,ENGL,CATHOLIC,WIDOWED,WHITE,SAH,F,2067-09-21,2151-03-06,2151-03-06,MICU,MICU,50,50,2151-03-05 20:01:18,2151-03-06 10:54:24,1,q0


# I-Diversity

In [ ]:
sensitive_variables = ['DIAGNOSIS']

In [ ]:
df_new_after_step_3 = naive_search(['LOS'], df)

if generalize on LOS we need to remove 99014 records.


In [ ]:
df_new_after_step_3.head()

,Unnamed: 0,SUBJECT_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,DIAGNOSIS,GENDER,DOB,DOD,DOD_HOSP,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,MORTALITY,LOS_percentile
0,0,22,2196-04-09 12:26:00,2196-04-10 15:54:00,-999,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,-999,UNOBTAINABLE,MARRIED,WHITE,BENZODIAZEPINE OVERDOSE,F,2131-05-07,-999,-999,MICU,MICU,52,52,2196-04-09 12:27:00,2196-04-10 15:54:00,0,q0
1,1,23,2153-09-03 07:15:00,2153-09-08 19:10:00,-999,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,-999,CATHOLIC,MARRIED,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,M,2082-07-17,-999,-999,CSRU,CSRU,14,14,2153-09-03 09:38:55,2153-09-04 15:59:11,0,q0
2,2,23,2153-09-03 07:15:00,2153-09-08 19:10:00,-999,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,-999,CATHOLIC,MARRIED,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,M,2082-07-17,-999,-999,SICU,SICU,57,57,2157-10-21 11:40:38,2157-10-22 16:08:48,0,q0
3,3,23,2157-10-18 19:34:00,2157-10-25 14:00:00,-999,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,BRAIN MASS,M,2082-07-17,-999,-999,CSRU,CSRU,14,14,2153-09-03 09:38:55,2153-09-04 15:59:11,0,q0
4,4,23,2157-10-18 19:34:00,2157-10-25 14:00:00,-999,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,BRAIN MASS,M,2082-07-17,-999,-999,SICU,SICU,57,57,2157-10-21 11:40:38,2157-10-22 16:08:48,0,q0


In [ ]:
for var in sensitive_variables:
    i_diversity = df_new_after_step_3[var].value_counts()
    print("sensitive variable name: "+var)
    print(i_diversity)
    print("=============================================")

sensitive variable name: DIAGNOSIS
NEWBORN                                 8049
PNEUMONIA                               4347
SEPSIS                                  3308
CONGESTIVE HEART FAILURE                2999
DIABETIC KETOACIDOSIS                   2807
                                        ... 
MALIGNANT CENTRAL AIRWAY OBSTRUCTION       1
MALIGNANT ACITES/SDA                       1
MALFUCTION ICD                             1
MALAISE S/P SLEEVE GASTRECTOMY             1
INTOXICATION;AGITATION                     1
Name: DIAGNOSIS, Length: 15669, dtype: int64


In [ ]:
new_quasi_lists = ['ADMISSION_TYPE', 'ADMISSION_LOCATION', 'DISCHARGE_LOCATION',
       'INSURANCE', 'LANGUAGE', 'RELIGION', 'MARITAL_STATUS', 'ETHNICITY','GENDER', 
        'FIRST_CAREUNIT', 'LAST_CAREUNIT', 'FIRST_WARDID', 'LAST_WARDID','LOS_percentile']

In [ ]:
# take l-diveristy for each group divided by new quasi list

grouped = df_new_after_step_3.groupby(by=new_quasi_lists)

l_diversity =  {"DIAGNOSIS":{}}

for name, group in grouped:
    for var in sensitive_variables:
        l_div = group[var].nunique()
        #number of unique value/I-diversity with counts 
        if l_div in l_diversity[var].keys():
            l_diversity[var][l_div] += 1
        else:
            l_diversity[var][l_div] = 1

In [ ]:
df_new_after_step_3

,Unnamed: 0,SUBJECT_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,DIAGNOSIS,GENDER,DOB,DOD,DOD_HOSP,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,MORTALITY,LOS_percentile
0,0,22,2196-04-09 12:26:00,2196-04-10 15:54:00,-999,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,-999,UNOBTAINABLE,MARRIED,WHITE,BENZODIAZEPINE OVERDOSE,F,2131-05-07,-999,-999,MICU,MICU,52,52,2196-04-09 12:27:00,2196-04-10 15:54:00,0,q0
1,1,23,2153-09-03 07:15:00,2153-09-08 19:10:00,-999,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,-999,CATHOLIC,MARRIED,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,M,2082-07-17,-999,-999,CSRU,CSRU,14,14,2153-09-03 09:38:55,2153-09-04 15:59:11,0,q0
2,2,23,2153-09-03 07:15:00,2153-09-08 19:10:00,-999,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,-999,CATHOLIC,MARRIED,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,M,2082-07-17,-999,-999,SICU,SICU,57,57,2157-10-21 11:40:38,2157-10-22 16:08:48,0,q0
3,3,23,2157-10-18 19:34:00,2157-10-25 14:00:00,-999,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,BRAIN MASS,M,2082-07-17,-999,-999,CSRU,CSRU,14,14,2153-09-03 09:38:55,2153-09-04 15:59:11,0,q0
4,4,23,2157-10-18 19:34:00,2157-10-25 14:00:00,-999,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,BRAIN MASS,M,2082-07-17,-999,-999,SICU,SICU,57,57,2157-10-21 11:40:38,2157-10-22 16:08:48,0,q0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116421,116421,98794,2127-11-07 11:00:00,2127-11-11 14:30:00,-999,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,AORTIC STENOSIS\AORTIC VALVE REPLACEMENT /SDA,M,2049-07-29,-999,-999,CSRU,CSRU,15,15,2127-11-07 10:02:54,2127-11-09 16:58:46,0,q0
116422,116422,98797,2132-12-24 20:06:00,2132-12-25 12:00:00,2132-12-25 12:00:00,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,Medicare,ENGL,CATHOLIC,WIDOWED,WHITE,ALTERED MENTAL STATUS,M,2044-12-27,2132-12-25,2132-12-25,MICU,MICU,50,50,2132-12-24 20:08:16,2132-12-26 01:51:14,1,q0
116423,116423,98800,2131-03-30 21:13:00,2131-04-02 15:02:00,-999,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Private,ENGL,NOT SPECIFIED,SINGLE,WHITE,TRAUMA,F,2111-11-05,-999,-999,TSICU,TSICU,14,14,2131-03-30 21:14:14,2131-03-31 18:18:14,0,q0
116424,116424,98802,2151-03-05 20:00:00,2151-03-06 09:10:00,2151-03-06 09:10:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,DEAD/EXPIRED,Medicare,ENGL,CATHOLIC,WIDOWED,WHITE,SAH,F,2067-09-21,2151-03-06,2151-03-06,MICU,MICU,50,50,2151-03-05 20:01:18,2151-03-06 10:54:24,1,q0


In [ ]:
l_diversity

{'DIAGNOSIS': {1: 39521,
  2: 5936,
  3: 2239,
  4: 1078,
  5: 632,
  6: 455,
  7: 296,
  8: 201,
  9: 166,
  10: 129,
  11: 87,
  12: 89,
  13: 61,
  14: 48,
  15: 39,
  16: 31,
  17: 22,
  18: 20,
  19: 16,
  20: 12,
  21: 9,
  22: 13,
  23: 12,
  24: 7,
  25: 5,
  26: 3,
  27: 3,
  28: 2,
  29: 3,
  30: 1,
  31: 2,
  32: 1,
  33: 1,
  35: 1,
  37: 1,
  39: 1,
  43: 1,
  44: 1}}

f we only look at minimal l-diversity:
them acheived only 1-diversity.

Overall: it achieved only 1-diversity, there are many diagnosis with only 1 individual record grouped by quasi identifier. This is also the reason that diagnosis is selected as one of our criteria to do blurring as we mentioned;

In [ ]:
[min(l_diversity[var].keys()) for var in sensitive_variables]

[1]

In [ ]:
sum(l_diversity["DIAGNOSIS"].values())

51145

Next, we take a look at the percentage of data that achieve 3-diversity or 5-diversity.

In [ ]:
def k_diversity_percentage(k=3):
    achieved_k_diversity = 0
    count = 0
    for i in l_diversity["DIAGNOSIS"]:
        count += l_diversity["DIAGNOSIS"][i]
        if i >= k:
            achieved_k_diversity += l_diversity["DIAGNOSIS"][i]
    return achieved_k_diversity/count

In [ ]:
k_diversity_percentage(3)

0.1112132173232965

In [ ]:
k_diversity_percentage(5)

0.04635839280477075